**LIBRARIES**

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from  matplotlib.pyplot import get_cmap
from sklearn.cluster import AgglomerativeClustering, SpectralClustering
from sklearn.metrics.pairwise import pairwise_distances
from networkx.algorithms.community.centrality import girvan_newman
from sklearn import metrics
from scipy.special import binom

In [ ]:
filename = 'file.gml'

**EVALUATION METRICS**

In [ ]:
def true_positive(true_communities, pred_communities):
  """
    Calculates true positive assignment, which means that similar members
    are assigned to the same communities.

    Args:
      true_communities (list of lists): List of true communities' lists.
      pred_communities (list of lists): List of predicted communities' lists.

    Returns:
      Value of true positive.
  """
  max_labels = []
  for pred_com in pred_communities:
    count0 = 0
    count1 = 0
    for node in pred_com:
      if node in true_communities[0]:
        count0 += 1
      else:
        count1 += 1
    max_labels.append(max(count0, count1))

  tp = binom(2, 2) + sum([binom(label, 2) for label in max_labels])
  return tp

def count_cluster_labels(true_communities, pred_communities):
  """
    Loops through the predicted communities and identifies the number of
    nodes that belongs to the corresponding true community.

    Args:
      true_communities (list of lists): List of true communities' lists.
      pred_communities (list of lists): List of predicted communities' lists.

    Returns:
      count_labels (list): List of the number of nodes
        which belong to the corresponding true community
  """
  count_labels = []
  for pred_com in pred_communities:
    count0 = 0
    count1 = 0
    for node in pred_com:
      if node in true_communities[0]:
        count0 += 1
      else:
        count1 += 1
    count_labels.append([count0, count1])
  return count_labels

def false_positive(true_communities, pred_communities):
  """
    Calculates false positive assignment, which means that dissimilar members
    are assigned to the same communities.

    Args:
      true_communities (list of lists): List of true communities' lists.
      pred_communities (list of lists): List of predicted communities' lists.

    Returns:
      Value of false positive.
  """

  count_labels = count_cluster_labels(true_communities, pred_communities)

  fp = 0
  for labels in count_labels:
    fp += math.prod(labels)
  return fp

def false_negative(true_communities, pred_communities):
  """
    Calculates false negative assignment, which means that similar members are
    assigned to different communities.

    Args:
      true_communities (list of lists): List of true communities' lists.
      pred_communities (list of lists): List of predicted communities' lists.

    Returns:
      Value of false negative.
  """
  count_labels = count_cluster_labels(true_communities, pred_communities)

  fn = 0
  for i in range(len(count_labels)):
    for j in range(len(count_labels[i])):
      fn += count_labels[i][j] * sum(count_labels[k][j] for k in range(i+1, len(count_labels)))
  return fn

In [ ]:
# Precision
def calc_precision(true_communities, pred_communities):
  """
    Calculates precision.

    Args:
      true_communities (list of lists): List of true communities' lists.
      pred_communities (list of lists): List of predicted communities' lists.

    Returns:
      Value of precision.
  """
  tp = true_positive(true_communities, pred_communities)
  fp = false_positive(true_communities, pred_communities)
  return tp / (tp + fp)

# Recall
  """
    Calculates recall.

    Args:
      true_communities (list of lists): List of true communities' lists.
      pred_communities (list of lists): List of predicted communities' lists.

    Returns:
      Value of recall.
  """
def calc_recall(true_communities, pred_communities):
  tp = true_positive(true_communities, pred_communities)
  fn = false_negative(true_communities, pred_communities)
  return tp / (tp + fn)

# Purity
def calc_purity(true_communities, pred_communities):
  """
    Calculates purity.

    Args:
      true_communities (list of lists): List of true communities' lists.
      pred_communities (list of lists): List of predicted communities' lists.

    Returns:
      Value of purity.
  """
  true_labels = [label for label, com in enumerate(true_communities) for _ in range(len(com))]
  pred_labels = [label for label, com in enumerate(pred_communities) for _ in range(len(com))]

  confusion_matrix = metrics.confusion_matrix(true_labels, pred_labels)

  return np.sum(np.amax(confusion_matrix, axis=0)) / np.sum(confusion_matrix)

In [ ]:
# Normalized Mutual Information
def calc_nmi(true_communities, pred_communities, num_nodes):
  """
    Calculates normalized mutual information.

    Args:
      true_communities (list of lists): List of true communities' lists.
      pred_communities (list of lists): List of predicted communities' lists.
      num_nodes (int): Number of nodes.

    Returns:
      Value of NMI.
  """

  def calc_prob(nodes, communities):
    """
    Calculates probability.

    Args:
      communities (list of lists): List of communities' lists.
      nodes (int): Number of nodes.

    Returns:
      Probability of class/cluster labels.
  """
    return [len(com) / nodes for com in communities]

  def calc_entropy(prob):
    """
    Calculates entropy.

    Args:
      prob (float): Probability of class/cluster labels.

    Returns:
      Entropy of class/cluster labels.
  """
    entropy_labels = 0
    for p in prob:
      entropy_labels -= p * math.log(p, 2)
    return entropy_labels

  # Entropy of Class Labels
  class_prob = calc_prob(num_nodes, true_communities)
  entropy_class_labels = calc_entropy(class_prob)

  # Entropy of Cluster Labels
  cluster_prob = calc_prob(num_nodes, pred_communities)
  entropy_cluster_labels = calc_entropy(cluster_prob)

  # Calculate conditional entropy of true labels for clustering
  prob_y_c = []
  for pred_com in pred_communities:
      save_prob_c_y = []
      for true_com in true_communities:
          same_class_cluster = len(set(true_com) & set(pred_com))
          save_prob_c_y.append(same_class_cluster / len(pred_com))
      prob_y_c.append(save_prob_c_y)

  entropy_y_c = 0
  for i, save_prob_c_y in enumerate(prob_y_c):
      entropy_c_y = 0
      for j, prob in enumerate(save_prob_c_y):
          if prob != 0:
              entropy_c_y += prob * math.log(prob, 2)
      entropy_y_c -= cluster_prob[i] * entropy_c_y

  # Calculate mutual information
  mutual_info = entropy_class_labels - entropy_y_c

  # Calculate normalized mutual information
  norm_mutual_info = (2 * mutual_info) / (entropy_class_labels + entropy_cluster_labels)

  return norm_mutual_info

In [ ]:
# Modularity
def calc_modularity(graph, communities):
  """
    Calculates modularity of the clustered graph.

    Args:
      graph : A (un)directed graph
      communities (list of lists): List of communities' lists

    Returns:
      Value of modularity.
  """
  return nx.algorithms.community.modularity(graph, communities)

# Conductance
def calc_conductance(graph, communities):
  """
    Calculates conductance of the clustered graph.


    Args:
      graph : A (un)directed graph
      communities (list of lists): List of communities' lists

    Returns:
      Value of conductance.
  """
  return min(nx.conductance(graph, com, weight='weight')
            for com in communities)

# Density
def calc_density(graph, communities):
  """
    Calculates density of the clustered graph.

    Args:
      graph : A (un)directed graph
      communities (list of lists): List of communities' lists

    Returns:
      Value of density.
  """
  list_density = []
  for com in communities:
    sub = graph.subgraph(com)
    list_density.append(nx.density(sub))
  return np.mean(list_density)

**PLOT FUNCTION**

In [ ]:
def plot_graph(graph, communities):

  """
    Plots the graph along with the communities detected.

    Args:
      graph : A (un)directed graph
      communities (list of lists): List of communities' lists

    Returns:
      -
  """

  colors = ['skyblue', 'lightcoral', 'limegreen', 'mediumpurple', 'hotpink', 'gold']
  pos = nx.spring_layout(graph)

  plt.figure(figsize=(20, 15))
  nx.draw_networkx_edges(graph, pos=pos, edge_color='darkgray')
  for community, color in zip(communities, colors):
      nx.draw_networkx_nodes(graph, pos=pos,
                            nodelist=community, node_color=color, node_size=250)
  nx.draw_networkx_labels(graph, pos=pos, font_size=9)
  _ = plt.axis('off')
  plt.show()

**OTHER FUNCTIONS**

In [ ]:
def create_list_of_communities(dict_labels):

  """
    Creates a list of lists given a dictionary of labels,
    where each inner list respresents a community.

    Args:
      dict_labels (dictionary): A dictionary of labels which has the form
          {key = nodes : value = label}. The labels' values are 0 or 1.

    Returns:
      list_communities (list of lists): List of communities' lists
  """

  # Create a list with two inner empty lists
  list_communities = [[],[]]
  # Filter nodes' label
  for node in dict_labels.keys():
    if dict_labels[node] == 0:
      # Community 0
      list_communities[0].append(node)
    elif dict_labels[node] == 1:
      # Community 1
      list_communities[1].append(node)
    else:
      continue

  return list_communities

# File's Graph

In [ ]:
# Read the directed graph from the .gml file
G = nx.read_gml(filename, label='id')

G_nodes = len(G.nodes())
G_edges = G.size()
print("* The original graph G:")
print("Nodes:", G_nodes)
print("Edges:", G_edges)
print()

# Remove duplicates of edges
H = nx.DiGraph(G)
# Find the largest connected component of the graph
H_lcc = max(nx.strongly_connected_components(H), key=len)
# Create a subgraph using the set of nodes from the largest component
H_subgraph = H.subgraph(H_lcc)

H_nodes = len(H_subgraph.nodes())
H_edges = H_subgraph.size()

print("* The largest connected component of directed G, turned into a subgraph:")
print("Nodes:", H_nodes)
print("Edges:", H_edges)
print()

# Convert subgraph to undirected
H_undirected_subgraph = H_subgraph.to_undirected()

# Get the 'value' attribute of each node
node_values = dict()
for node in H_subgraph.nodes():
  node_values[node] = H_subgraph.nodes[node]['value']

print("* The ground-truth values of each node:\n", node_values)
print()

# Keep the nodes of each community into a list of lists
true_communities = create_list_of_communities(node_values)
print("* The ground-truth communities are:\n", true_communities)

**Performance Evaluation**

In [ ]:
print("* Evaluation of ground-truth communities:")
print("Modularity:", calc_modularity(H_undirected_subgraph, true_communities))
print("Conductance:", calc_conductance(H_undirected_subgraph, true_communities))
print("Density:", calc_density(H_undirected_subgraph, true_communities))

**Visualization (Ground Truth Plot)**

In [ ]:
plot_graph(H_subgraph, true_communities)

# 1. Clauset-Newman-Moore Greedy Modularity Maximization

In [ ]:
greedy_communities = list(nx.algorithms.community.greedy_modularity_communities(H_undirected_subgraph))

**Performance Evaluation**

In [ ]:
print("* Evaluation of Greedy Modularity Maximization communities:")
print("Precision:", calc_precision(true_communities, greedy_communities))
print("Recall:", calc_recall(true_communities, greedy_communities))
print("Purity:", calc_purity(true_communities, greedy_communities))
print("NMI:", calc_nmi(true_communities, greedy_communities, H_nodes))
print("Modularity:", calc_modularity(H_undirected_subgraph, greedy_communities))
print("Conductance:", calc_conductance(H_undirected_subgraph, greedy_communities))
print("Density:", calc_density(H_undirected_subgraph, greedy_communities))

**Visualization**

In [ ]:
plot_graph(H_subgraph, greedy_communities)

# 2. Agglomerative Hierarchical Clustering

In [ ]:
# Number of clusters
n_clusters = 2

# Get the adjacency matrix from the graph
adj_matrix = nx.to_numpy_array(H_undirected_subgraph)

In [ ]:
# Apply agglomerative clustering
model = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward')
pred_labels = model.fit_predict(adj_matrix)
pred_communities = {i : community for i, community in zip(H_subgraph.nodes(), pred_labels)}

**Performance Evaluation**

In [ ]:
agg_communities = create_list_of_communities(pred_communities)

In [ ]:
print("* Evaluation of Agglomerative clustering:")
print("Precision:", calc_precision(true_communities, agg_communities))
print("Recall:", calc_recall(true_communities, agg_communities))
print("Purity:", calc_purity(true_communities, agg_communities))
print("NMI:", calc_nmi(true_communities, agg_communities, H_nodes))
print("Modularity:", calc_modularity(H_undirected_subgraph, agg_communities))
print("Conductance:", calc_conductance(H_undirected_subgraph, agg_communities))
print("Density:", calc_density(H_undirected_subgraph, agg_communities))

**Visualization**

In [ ]:
plot_graph(H_subgraph, agg_communities)

# 3. Divisive Hierarchical Method

(based on Albert-László Barabási, http://networksciencebook.com/)

In [ ]:
def girvan_newman(G):
    """ run the algorithm of Girvan + Newman up to the first separation
        and return list of components of G, list of edges removed
    """

    # we're going to remove edges, so do it on a copy of the original graph
    G = G.copy()

    def find_best_edge(G0):
        """ get the edge from G0 with highest betweenness centrality"""
        eb = nx.edge_betweenness_centrality(G0)
        edges = eb.keys()
        return max(edges, key=lambda e: eb[e])

    removed_edges = []
    # Proceed until we separate the graph
    while nx.number_connected_components(G) == 1:
        u, v = find_best_edge(G)
        G.remove_edge(u, v)
        removed_edges.append((u, v))

    return list(nx.connected_components(G)), removed_edges

In [ ]:
gn_communities, removed_edges = girvan_newman(H_undirected_subgraph)
other_edges = set(H_subgraph.edges()) - set(removed_edges)

**Performance Evaluation**

In [ ]:
print("* Evaluation of Girvan-Newman algorithm:")
print("Precision:", calc_precision(true_communities, gn_communities))
print("Recall:", calc_recall(true_communities, gn_communities))
print("Purity:", calc_purity(true_communities, gn_communities))
print("NMI:", calc_nmi(true_communities, gn_communities, H_nodes))
print("Modularity:", calc_modularity(H_undirected_subgraph, gn_communities))
print("Conductance:", calc_conductance(H_undirected_subgraph, gn_communities))
print("Density:", calc_density(H_undirected_subgraph, gn_communities))

**Visualization**

In [ ]:
plot_graph(H_subgraph, gn_communities)

# 4. Spectral Clustering

In [ ]:
# Apply spectral clustering to the laplacian matrix
model = SpectralClustering(n_clusters = n_clusters, affinity='precomputed')
pred_labels = model.fit_predict(adj_matrix)
pred_communities = {i : community for i, community in zip(H_subgraph.nodes(), pred_labels)}

**Performance Evaluation**

In [ ]:
spec_communities = create_list_of_communities(pred_communities)

In [ ]:
print("* Evaluation of Spectral Clustering:")
print("Precision:", calc_precision(true_communities, spec_communities))
print("Recall:", calc_recall(true_communities, spec_communities))
print("Purity:", calc_purity(true_communities, spec_communities))
print("NMI:", calc_nmi(true_communities, spec_communities, H_nodes))
print("Modularity:", calc_modularity(H_undirected_subgraph, spec_communities))
print("Conductance:", calc_conductance(H_undirected_subgraph, spec_communities))
print("Density:", calc_density(H_undirected_subgraph, spec_communities))

**Visualization**

In [ ]:
plot_graph(H_subgraph, spec_communities)